# AI-Book-Recommender: Model Evaluation
This notebook evaluates the SVD-based collaborative filtering model using RMSE on a small sample of the ratings dataset.

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from math import sqrt
from scipy.sparse.linalg import svds

# Load cleaned ratings dataset
ratings = pd.read_csv('ratings.csv')

In [ ]:
def evaluate_split_small(ratings, sample_size=1000):
    sample = ratings.sample(n=sample_size, random_state=42)
    train, test = train_test_split(sample, test_size=0.2, random_state=42)
    common_users = list(set(train['User-ID']) & set(test['User-ID']))
    
    if len(common_users) == 0:
        print('No common users between train and test. Increase sample size.')
        return
    
    train = train[train['User-ID'].isin(common_users)]
    test = test[test['User-ID'].isin(common_users)]
    
    train_matrix = train.pivot(index='User-ID', columns='ISBN', values='Book-Rating').fillna(0)
    if train_matrix.shape[0] < 2 or train_matrix.shape[1] < 2:
        print('Not enough data for SVD.')
        return
    
    user_ratings_mean = train_matrix.mean(axis=1)
    R_demeaned = train_matrix.sub(user_ratings_mean, axis=0)
    U, sigma, Vt = svds(R_demeaned, k=50)
    sigma = np.diag(sigma)
    predicted_ratings = np.dot(np.dot(U, sigma), Vt) + user_ratings_mean.values.reshape(-1, 1)
    preds_df = pd.DataFrame(predicted_ratings, columns=train_matrix.columns, index=train_matrix.index)
    
    actual = []
    predicted = []
    for _, row in test.iterrows():
        user = row['User-ID']
        isbn = row['ISBN']
        if user in preds_df.index and isbn in preds_df.columns:
            actual.append(row['Book-Rating'])
            predicted.append(preds_df.loc[user, isbn])
    
    if len(actual) > 0:
        rmse = sqrt(mean_squared_error(actual, predicted))
        print(f'Actual RMSE (small sample): {rmse}')
    else:
        print('No valid predictions to compute RMSE.')

In [ ]:
# Run evaluation
evaluate_split_small(ratings, sample_size=1000)